## Setup


In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import random
import io


## Prepare the data


In [2]:
booklist = [["TreasureIsland.txt","https://www.gutenberg.org/cache/epub/120/pg120.txt"],['The_Adventures_of_Ferdinand_Count_Fathom.txt', 'https://www.gutenberg.org/cache/epub/6761/pg6761.txt']]

In [3]:

path = keras.utils.get_file(
    booklist[0][0], origin=booklist[0][1]
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
    text = text.replace("\n", " ")  # We remove newlines chars for nicer display
    print("Corpus length:", len(text))



chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



399527/399527 [==============================] - 1s 2us/step
Corpus length: 383560
Total chars: 63
Number of sequences: 127840


<ipython-input-3-2ffe1f178dc1>:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-3-2ffe1f178dc1>:27: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


## Build the model: a single LSTM layer


In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)

# Loads the weights

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


## Prepare the text sampling function


In [5]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



## Train the model


In [14]:
!rm -r /content/training_1

In [13]:
import os
epochs = 40
batch_size = 128

checkpoint_path = "/content/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


for epoch in range(epochs):
    if epoch in [1,5,10,20,30,40]:
        save_weight_bool = False
    else:
        save_weight_bool = True

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=save_weight_bool,
                                                 verbose=1)

    if os.path.exists('/content/training_1/checkpoint'):
        print('loading weights!')
        model.load_weights(checkpoint_path)
    
    model.fit(x, y, batch_size=batch_size, epochs=3,callbacks=[cp_callback])
    print()
    print("Generating text after epoch: %d" % epoch)
    if epoch in [39]:
        start_index = random.randint(0, len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print("...Diversity:", diversity)

            generated = ""
            sentence = text[start_index : start_index + maxlen]
            print('...Generating with seed: "' + sentence + '"')

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.0
                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]
                sentence = sentence[1:] + next_char
                generated += next_char

            print("...Generated: ", generated)
            print()


loading weights!
Epoch 1/3
991/999 [============================>.] - ETA: 0s - loss: 1.0257
Epoch 1: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 1.0256
Epoch 2/3
994/999 [============================>.] - ETA: 0s - loss: 1.0282
Epoch 2: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 6s 6ms/step - loss: 1.0278
Epoch 3/3
990/999 [============================>.] - ETA: 0s - loss: 1.0247
Epoch 3: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 1.0252

Generating text after epoch: 0
loading weights!
Epoch 1/3
991/999 [============================>.] - ETA: 0s - loss: 1.0248
Epoch 1: saving model to /content/training_1/cp.ckpt


999/999 [==============================] - 9s 9ms/step - loss: 1.0250
Epoch 2/3
993/999 [============================>.] - ETA: 0s - loss: 1.0232
Epoch 2: saving model to /content/training_1/cp.ckpt


999/999 [==============================] - 9s 9ms/step - loss: 1.0233
Epoch 3/3
993/999 [============================>.] - ETA: 0s - loss: 1.0282
Epoch 3: saving model to /content/training_1/cp.ckpt


999/999 [==============================] - 10s 10ms/step - loss: 1.0284

Generating text after epoch: 1
loading weights!
Epoch 1/3
990/999 [============================>.] - ETA: 0s - loss: 1.0233
Epoch 1: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 1.0235
Epoch 2/3
998/999 [============================>.] - ETA: 0s - loss: 1.0222
Epoch 2: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 6s 6ms/step - loss: 1.0222
Epoch 3/3
993/999 [============================>.] - ETA: 0s - loss: 1.0211
Epoch 3: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 1.0214

Generating text after epoch: 2
loading weights!
Epoch 1/3
998/999 [============================>.] - ETA: 0s - loss: 1.0248
Epoch 1: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 1.0248
Epoch 2/3
999/999 [===============

999/999 [==============================] - 9s 9ms/step - loss: 1.0182
Epoch 2/3
995/999 [============================>.] - ETA: 0s - loss: 1.0146
Epoch 2: saving model to /content/training_1/cp.ckpt


...

999/999 [==============================] - 9s 9ms/step - loss: 0.9666
Epoch 2/3
995/999 [============================>.] - ETA: 0s - loss: 0.9668
Epoch 2: saving model to /content/training_1/cp.ckpt


999/999 [==============================] - 8s 8ms/step - loss: 0.9669
Epoch 3/3
995/999 [============================>.] - ETA: 0s - loss: 0.9663
Epoch 3: saving model to /content/training_1/cp.ckpt


999/999 [==============================] - 10s 10ms/step - loss: 0.9667

Generating text after epoch: 30
loading weights!
Epoch 1/3
997/999 [============================>.] - ETA: 0s - loss: 0.9673
Epoch 1: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 6s 6ms/step - loss: 0.9673
Epoch 2/3
997/999 [============================>.] - ETA: 0s - loss: 0.9672
Epoch 2: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 0.9670
Epoch 3/3
998/999 [============================>.] - ETA: 0s - loss: 0.9621
Epoch 3: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 0.9620

Generating text after epoch: 31
loading weights!
Epoch 1/3
990/999 [============================>.] - ETA: 0s - loss: 0.9613
Epoch 1: saving model to /content/training_1/cp.ckpt
999/999 [==============================] - 5s 5ms/step - loss: 0.9614
Epoch 2/3
993/999 [=============

<ipython-input-5-0d226062c96c>:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


...Generated:  etund the doct. ‘block himself was the track and some for his face is no me a man to take to see himber shake; and if you mare had heard a chatch of the coast of the green to have the tall flashes of the doctor’s longer came to the shupies:                             pow of the man the money again to the secres.  “for, when you say! and i’m been some mone, and i’ll tell you the cook came to the m

...Diversity: 1.0
...Generating with seed: "drinks?” cried the squire.  “no, sir,” r"
...Generated:  etund gaved he cont; and the more of the conshor is that he had not uuse, on the other bars of priniatleh and light man’s ebering good expose, not another never metime, surrling merciles the fores and some it, before you mark-qusted maybecuances reply.”  “the,” said the captain.  “now, you’d be dangeons, steps?”  “is that many bearger for it!” said me in the old phokes thank. now, of this cruise. 

...Diversity: 1.2
...Generating with seed: "drinks?” cried the squire.  “no, sir

In [14]:
from google.colab import files
!zip -r /content/training_1.zip /content/training_1
files.download('/content/training_1.zip')

  adding: content/training_1/ (stored 0%)
  adding: content/training_1/cp.ckpt.data-00000-of-00001 (deflated 7%)
  adding: content/training_1/cp.ckpt.index (deflated 51%)
  adding: content/training_1/cp.ckpt/ (stored 0%)
  adding: content/training_1/cp.ckpt/variables/ (stored 0%)
  adding: content/training_1/cp.ckpt/variables/variables.index (deflated 51%)
  adding: content/training_1/cp.ckpt/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/training_1/cp.ckpt/keras_metadata.pb (deflated 85%)
  adding: content/training_1/cp.ckpt/saved_model.pb (deflated 91%)
  adding: content/training_1/cp.ckpt/fingerprint.pb (stored 0%)
  adding: content/training_1/cp.ckpt/assets/ (stored 0%)
  adding: content/training_1/checkpoint (deflated 38%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
def predict(_model, seed, diversity=0.5, NrOffChars=1000):
  
    generated = ""
    sentence = seed
    print('...Generating with seed: "' + sentence + '"')

    for i in range(NrOffChars):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = _model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    print("...Generated: ", generated)
    print()


predict(model, "hello u stupid ai", 0.5, 1000)

...Generating with seed: "hello u stupid ai"
...Generated:  umbwbywbmcmmycfbwumcmummmbmwyfccybbwbmwymffbpwmymbcpbmybmuwbfbwbmmwmymmibbbwcpwyyybbcbcwmbwfmwbubwbwwwymmfypawwybmbbmbwwmmbbbmwubfmywuwmwbculwmysbwmbmymcwmumbmwbcmbcmbcmbnymbwbcbymyymffbbbmmpwbmmbmimcmbywswbmbbmbbbbbcbmbbcmfmmbbmbcsbfmcmbfbymwbbfmmwwmmomaynmbybypbwymmbmbbmmmwmymmbrpywaymswufbbmbcbbyymmbmywcsplybbwmcbwmmmwnmpwwbbmmwoycnwmbmwmybmwbmysyywmsmibccybbymmbcmbybwmymmchmwmwmpbmbmmmcybwfmsmmwwfmwmwmbbmbccmsmwcbnbmmbcmbbbmyfcmwpusmbmbmymbybyybcmywmycmfwmwbwgwpmymybcybwmbfbbwmcmyyiymgpbmbyyymlbcmfmmwwbmmgmybmmbwywclmbmmbcuyyrwfblmmmgmwbwbccbmbrbbbmmacuybbwybmywmuymbmwfmmbbmbcbmwsybbwwbyycdbfbfbywsbbwwywbfybymmywmymbbfmmwycmmrmlypwmbmwcmbbmasfwbmwcmycibpbmbmmlmrsbsmabmblsbmwbmbwbcbwbysnsbblbmbbpumncwbbpbwcwbbbbcbmmmwprccybmypyubpwbwcfwmmwmmwwmmmwwywwmbywfmywmbwwibmlwwcycsmwbbwmwblmmmfmbcbpbmmmsmwbbbyh.mbwlwwbbmmafmbbbbbbcmfbmbmamwwmmywwbbbyyybyyymmlbmwmmbmmbfybcbwuwbbycyybywmwmbycnsmbmblimwbybcmcwbbwybybfwbbbfvmbcbsbwclwpi